First part - add geolocation information to PNG files:

In [7]:
import rasterio
import os
import shutil

from rasterio.merge import merge
from rasterio.plot import show


In [5]:
# This part of the script stitches pngs located in aut_dir, based on georeferenced images in tif_dir.
# Matching is done based on file names.
png_dir = './spring_2018/'
tif_dir = '../to_predict/combined_tifs/'

png_dirs = [png_dir+x+'/' for x in os.listdir(png_dir) if '.DS_Store' not in x]

for png_dir in png_dirs:
    png_list = [x for x in os.listdir(png_dir) if x[-3:] == 'png']
    tif_list = ['_'.join(x.split('_')[0:4])+'.tif' for x in png_list if x[-3:] == 'png']

    # Add geolocation data to PNGs from TIFs
    for i,tif in enumerate(tif_list):
        src = rasterio.open(tif_dir+tif)

        input_crs = src.crs
        input_transform = src.transform

        with rasterio.open(png_dir+png_list[i], 'r+') as rds:
            rds.crs = input_crs
            rds.transform = input_transform
    
    # Create the mosaic
    src_files_to_mosaic = []

    for file in png_list:
        source = rasterio.open(png_dir+file)
        src_files_to_mosaic.append(source)

    mosaic, out_trans = merge(src_files_to_mosaic)
        # Save to file

    with rasterio.open(
            png_dir[:-1]+".png",
            mode="w",
            driver="PNG",
            dtype=mosaic.dtype,
            height=mosaic.shape[1],
            width=mosaic.shape[2],
            count=3,
            crs=src.crs,
            transform=out_trans
    ) as new_dataset:
            new_dataset.write(mosaic, [1,2,3])
        

/Users/michaelurson/miniconda3/envs/ssml/lib/python3.10/site-packages/rasterio/__init__.py:306: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = get_writer_for_path(path, driver=driver)(


This part is to visualize testing vs training dataset:

In [3]:
# Split files into folders for easier use:
aut_final = './spring_2018/'
seasons = list(set([x.split('_')[0] for x in os.listdir(aut_final)]))
pngs = [x for x in os.listdir(aut_final) if x[-3:] == 'png']

for season in seasons:
    os.mkdir(aut_final+season)
    for png in pngs:
        if season in png:
            shutil.move(aut_final+png,aut_final+season+'/'+png)
